In [ ]:
from fpylll import IntegerMatrix,LLL
from sage.all import *
import sys
#ref https://github.com/mimoo/RSA-and-LLL-attacks/blob/master/coppersmith.sage

In [ ]:
N=84364443735725034864402554533826279174703893439763343343863260342756678609216895093779263028809246505955647572176682669445270008816481771701417554768871285020442403001649254405058303439906229201909599348669565697534331652019516409514800265887388539283381053937433496994442146419682027649079704982600857517093
e=5
C=75352572919099024440173779584547333497560043131603516699957112836410957015813126201563840411306074334782444417011834059318772224061708608817009588365705592425310770025754431011094376930101106797532506390499679206979537302529273664034909312465679259526346126162190955050086991526496872886314055073606819188206

In [ ]:
res = None
def break_RSA(padding, len_p):
    global res
    bin_padding = ""
    ZmodN = Zmod(N);
    for x in padding:
        bin_padding+= str(bin(ord(x))[2:].zfill(8))

    beta = 1
    eps = beta / 7

    for length_M in range(0, len_p+1, 8): #rotating the bits of padding 8 bits at a time
        P.<M> = PolynomialRing(ZmodN) #here M is the message.Takes coefficient mod N
        polynomial = ((int(bin_padding, 2)<<length_M) + M)^e - C
        m = ceil(1**2 / (polynomial.degree() * eps)) # beta = 1, eps = 1/7
        val = floor(polynomial.degree() * m * ((1/beta) - 1))
        X = ceil(N**((1**2/polynomial.degree()) - eps))

        roots = coppersmith_howgrave_univariate(polynomial,  beta, m, val, X) # Calling function coppersmith to get the roots

        if roots:
            print("The root is :", bin(roots[0])[2:])
            res = bin(roots[0])[2:]
            return

    print('No Solution found')

def coppersmith_howgrave_univariate(polynomial, beta, m, val, X):
    poly_list = [] # stores Polynomials
    polynomial = polynomial.change_ring(ZZ)
    degree = polynomial.degree()
    x = polynomial.change_ring(ZZ).parent().gen()
   
    for i in range(m):
        for j in range(degree):
            poly_list.extend([(x * X)**j * N**(m - i) * polynomial(x * X)**i])
    for i in range(val):
        poly_list.extend([(x * X)**i * polynomial(x * X)**m]) #construct polynomials

    latice_mat_dim = degree * m + val
    lattice_ = Matrix(ZZ, latice_mat_dim) # creating lattice to find the shortest vector

    for i in range(latice_mat_dim):
        for j in range(i+1):
            lattice_[i, j] = poly_list[i][j]

    lattice_ = lattice_.LLL() # LLL

    polynomial = 0
    for i in range(latice_mat_dim):
        polynomial += x**i * lattice_[0, i] / X**i # shortest vector

    possible_roots = polynomial.roots() # Stores the possible roots

    roots_list = [] # true roots
    for root in possible_roots:
        if root[0].is_integer():
            result = polynomial(ZZ(root[0]))
            if(gcd(N, result) >= N^beta):
                roots_list.extend([ZZ(root[0])])

    return roots_list





break_RSA("Sleuth_Society: This door has RSA encryption with exponent 5 and the password is", 250)

In [ ]:
password=""
if(len(res)%8!=0):
    diff = len(res)%8
    res = res[diff:]

for i in range( 0, len(res), 8):
    password += chr( int( res[i:i+8] , 2) )
print(password)